In [1]:
import logging, json, time, os, random
from logging.handlers import RotatingFileHandler

 
LOG_FILE = os.getenv("BOT_LOG", "bot.log")
logger = logging.getLogger("binance_bot")
logger.setLevel(logging.INFO)
if not logger.handlers:
    fh = RotatingFileHandler(LOG_FILE, maxBytes=5_000_000, backupCount=3)
    fh.setFormatter(logging.Formatter("%(message)s"))
    logger.addHandler(fh)

def log_event(level: str, action: str, payload: dict):
    entry = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "level": level,
        "action": action,
        "payload": payload
    }
    logger.info(json.dumps(entry))
 
class DummyClient:
    def __init__(self):
        self._exchange_info = {
            "symbols": [{
                "symbol": "BTCUSDT",
                "filters": [
                    {"filterType": "PRICE_FILTER", "minPrice": "0.01", "maxPrice": "1000000", "tickSize": "0.01"},
                    {"filterType": "LOT_SIZE", "minQty": "0.0001", "maxQty": "1000", "stepSize": "0.0001"}
                ]
            }]
        }

    def futures_create_order(self, **kwargs):
        return {
            "symbol": kwargs["symbol"],
            "type": kwargs["type"],
            "side": kwargs["side"],
            "orderId": random.randint(100000, 999999),
            "status": "NEW",
            "quantity": kwargs["quantity"],
            "price": kwargs.get("price", None),
            "time": int(time.time()*1000)
        }

    def futures_exchange_info(self):
        return self._exchange_info
 
def get_client(simulate=True):
    return DummyClient()   

def _find_symbol_info(exchange_info, symbol):
    for s in exchange_info.get("symbols", []):
        if s["symbol"].upper() == symbol.upper():
            return s
    return None

def validate_order_inputs(symbol, side, quantity, simulate=True):
    if side.upper() not in ("BUY", "SELL"):
        raise ValueError("Side must be BUY or SELL")
    if quantity <= 0:
        raise ValueError("Quantity must be > 0")
    client = get_client(simulate)
    info = client.futures_exchange_info()
    sym = _find_symbol_info(info, symbol)
    if not sym:
        raise ValueError("Invalid symbol")
    return True

def validate_limit_order(symbol, side, quantity, price, simulate=True):
    if price <= 0: raise ValueError("Price must be > 0")
    validate_order_inputs(symbol, side, quantity, simulate)
    return True
 
def place_market_order(symbol, side, quantity, simulate=True):
    validate_order_inputs(symbol, side, quantity, simulate)
    client = get_client(simulate)
    resp = client.futures_create_order(symbol=symbol, side=side, type="MARKET", quantity=quantity)
    log_event("INFO", "MARKET_ORDER_SUCCESS", resp)
    return resp

def place_limit_order(symbol, side, quantity, price, simulate=True):
    validate_limit_order(symbol, side, quantity, price, simulate)
    client = get_client(simulate)
    resp = client.futures_create_order(symbol=symbol, side=side, type="LIMIT", quantity=quantity, price=price)
    log_event("INFO", "LIMIT_ORDER_SUCCESS", resp)
    return resp
 
def place_stop_limit(symbol, side, quantity, stop_price, limit_price, simulate=True):
    log_event("INFO", "STOP_LIMIT_INIT", {"symbol": symbol, "side": side, "stop_price": stop_price, "limit_price": limit_price})
    if simulate:
        log_event("INFO", "STOP_LIMIT_SIM_TRIGGER", {"message": "Triggered immediately (simulation mode)."})
        return place_limit_order(symbol, side, quantity, limit_price, simulate=True)
    raise NotImplementedError("Real stop-limit monitoring not implemented in this demo.")

def place_oco(symbol, side, quantity, take_profit, stop_loss, simulate=True):
    log_event("INFO", "OCO_INIT", {"symbol": symbol, "tp": take_profit, "sl": stop_loss})
    opposite_side = "SELL" if side.upper() == "BUY" else "BUY"
    tp_order = place_limit_order(symbol, opposite_side, quantity, take_profit, simulate)
    sl_order = place_limit_order(symbol, opposite_side, quantity, stop_loss, simulate)
    log_event("INFO", "OCO_COMPLETED", {"tp_order": tp_order, "sl_order": sl_order})
    return {"take_profit_order": tp_order, "stop_loss_order": sl_order}

def twap_order(symbol, side, total_qty, parts, interval_sec, simulate=True):
    qty_per_part = total_qty / parts
    results = []
    log_event("INFO", "TWAP_START", {"symbol": symbol, "side": side, "total_qty": total_qty, "parts": parts})
    for i in range(parts):
        result = place_market_order(symbol, side, qty_per_part, simulate)
        results.append(result)
        if i < parts - 1:
            time.sleep(interval_sec)
    log_event("INFO", "TWAP_DONE", {"symbol": symbol, "total_orders": len(results)})
    return results

def create_grid(symbol, base_price, lower, upper, levels, qty_per_level, simulate=True):
    log_event("INFO", "GRID_START", {"symbol": symbol, "lower": lower, "upper": upper, "levels": levels})
    step = (upper - lower) / max(1, levels - 1)
    orders = []
    for i in range(levels):
        price = round(lower + i * step, 2)
        buy = place_limit_order(symbol, "BUY", qty_per_level, price, simulate)
        sell = place_limit_order(symbol, "SELL", qty_per_level, price + step, simulate)
        orders.append({"level": i+1, "buy": buy, "sell": sell})
    log_event("INFO", "GRID_DONE", {"levels": levels})
    return orders


In [2]:
symbol = "BTCUSDT"

 
print("=== Market Order ===")
print(place_market_order(symbol, "BUY", 0.001, simulate=True))

print("\n=== Limit Order ===")
print(place_limit_order(symbol, "SELL", 0.001, 60000, simulate=True))

 
print("\n=== Stop-Limit Order ===")
print(place_stop_limit(symbol, "BUY", 0.001, stop_price=58000, limit_price=57900, simulate=True))

print("\n=== OCO Order ===")
print(place_oco(symbol, "BUY", 0.001, take_profit=61000, stop_loss=59000, simulate=True))

print("\n=== TWAP Order ===")
print(twap_order(symbol, "BUY", total_qty=0.005, parts=3, interval_sec=1, simulate=True))

print("\n=== Grid Orders ===")
print(create_grid(symbol, base_price=60000, lower=59500, upper=60500, levels=3, qty_per_level=0.001, simulate=True))

print("\n✅ All demo orders executed in simulation mode. Check bot.log for logs.")



=== Market Order ===
{'symbol': 'BTCUSDT', 'type': 'MARKET', 'side': 'BUY', 'orderId': 208868, 'status': 'NEW', 'quantity': 0.001, 'price': None, 'time': 1759661085350}

=== Limit Order ===
{'symbol': 'BTCUSDT', 'type': 'LIMIT', 'side': 'SELL', 'orderId': 855295, 'status': 'NEW', 'quantity': 0.001, 'price': 60000, 'time': 1759661085353}

=== Stop-Limit Order ===
{'symbol': 'BTCUSDT', 'type': 'LIMIT', 'side': 'BUY', 'orderId': 445813, 'status': 'NEW', 'quantity': 0.001, 'price': 57900, 'time': 1759661085354}

=== OCO Order ===
{'take_profit_order': {'symbol': 'BTCUSDT', 'type': 'LIMIT', 'side': 'SELL', 'orderId': 239560, 'status': 'NEW', 'quantity': 0.001, 'price': 61000, 'time': 1759661085354}, 'stop_loss_order': {'symbol': 'BTCUSDT', 'type': 'LIMIT', 'side': 'SELL', 'orderId': 184437, 'status': 'NEW', 'quantity': 0.001, 'price': 59000, 'time': 1759661085355}}

=== TWAP Order ===
[{'symbol': 'BTCUSDT', 'type': 'MARKET', 'side': 'BUY', 'orderId': 686678, 'status': 'NEW', 'quantity': 0.0